In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.optim import SGD,Adam,lr_scheduler

import torchvision
import torchvision.transforms as transforms

import os

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
print(device)

cuda


In [ ]:
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30), # Augmentation step
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

==> Preparing data..


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


In [ ]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        dropout=0.25
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.dropout = nn.Dropout(dropout)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out = self.dropout(out)
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [ ]:
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

In [ ]:
net = ResNet18()

In [ ]:
net.load_state_dict(torch.load('/content/drive/MyDrive/resnet_weights/saved_weights.pt'))

<All keys matched successfully>

In [ ]:
net.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (dropout): Dropout(p=0.25, inplace=False)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): 

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(),lr=0.001,
                      momentum=0.9, weight_decay=5e-4)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100)

In [ ]:
pred_list=[]
#batch_size=4
#test_loader = torch.utils.data.DataLoader(test_data, batch_size=4,  num_workers=0)
for data,labels in testloader:
    # move tensors to GPU if CUDA is available
    #if train_on_gpu:
        #data = data.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output=net(data)
    _,pred=torch.max(output,1)
    pred_list.extend(pred.data.cpu().detach().numpy())

In [ ]:
l=[]
for data,labels in testloader:
  l.append(labels)

In [ ]:
for i in range(len(l)):
  l[i]=list(l[i].numpy())

In [ ]:
f=[]
for i in range(len(l)):
  for j in l[i]:
    f.append(j)

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix
c=confusion_matrix(np.array(f),np.array(pred_list))

## Confusion Matrix

In [ ]:
c

array([[860,  16,  32,  13,   9,   4,   4,   4,  34,  24],
       [  1, 960,   2,   1,   0,   0,   0,   1,   6,  29],
       [ 13,   1, 865,  14,  40,  13,  30,  13,   2,   9],
       [  6,   2,  35, 762,  35, 102,  24,  20,   6,   8],
       [  6,   2,  18,  14, 916,  12,   8,  20,   3,   1],
       [  1,   2,  20,  75,  21, 850,   5,  22,   1,   3],
       [  3,   3,  11,  14,  10,   8, 947,   1,   0,   3],
       [  3,   0,   6,   5,  19,  20,   1, 935,   2,   9],
       [ 21,   8,   2,   0,   0,   2,   1,   1, 951,  14],
       [  7,  41,   2,   1,   1,   0,   2,   1,  10, 935]])

In [ ]:
from sklearn.metrics import accuracy_score
accu=accuracy_score(np.array(pred_list),np.array(f))

In [ ]:
cla=['aeroplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

In [ ]:
l=[]
for i in range(0,10):
  l.append(f'class: {cla[i]}: {c[i][i]/np.sum(c[i])}')


# Per Class Accuracy

In [ ]:
l

['class: aeroplane: 0.86',
 'class: automobile: 0.96',
 'class: bird: 0.865',
 'class: cat: 0.762',
 'class: deer: 0.916',
 'class: dog: 0.85',
 'class: frog: 0.947',
 'class: horse: 0.935',
 'class: ship: 0.951',
 'class: truck: 0.935']

# Inference
## cats and dogs are missclassifying maximum.
## cats are missclassifying as dogs and dogs as cats
## ships and automobile have highest accuracy among all classes
